# 智能语音助手实现

In [4]:
# -*- encoding: utf-8 -*-
import hashlib
import hmac
import base64
import json
import time
import threading
import re
from websocket import create_connection
from urllib.parse import quote
import logging
import pyaudio
import requests
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import TTS
from urllib.parse import urlencode
# 实时语音转写配置  
RTASR_APP_ID = "a4ab676a"  
RTASR_API_KEY = "c0a8020c200794a61aaa108740ab170c" 

# 沉默时间阈值（秒）
SILENCE_THRESHOLD = 3  

class Client:
    def __init__(self):
        # 初始化实时语音转写 WebSocket
        self.rtasr_ws = None
        self.setup_rtasr()

        # 用于存储转录文本和时间戳
        self.text_buffer = []  # 存储文本片段列表
        self.last_text_time = time.time()  # 上次接收文本的时间
        self.current_segment = ""  # 当前句子片段
        
        # 初始化大模型
        try:
            self.llm = Ollama(model="qwen2.5:7b-instruct", temperature=0.5)
            self.memory = ConversationBufferMemory()
            self.conversation = ConversationChain(llm=self.llm, memory=self.memory)
            print("本地大模型初始化成功")
        except Exception as e:
            print(f"大模型初始化失败: {e}")
            self.llm = None

        # 启动接收线程
        self.trecv_rtasr = threading.Thread(target=self.recv_rtasr)
        self.trecv_rtasr.daemon = True
        self.trecv_rtasr.start()

        # 启动定时检查线程
        self.tcheck = threading.Thread(target=self.check_silence)
        self.tcheck.daemon = True  # 设置为守护线程，随主线程结束
        self.tcheck.start()

    def setup_rtasr(self):
        """设置实时语音转写 WebSocket 连接"""
        base_url = "ws://rtasr.xfyun.cn/v1/ws"
        ts = str(int(time.time()))
        tt = (RTASR_APP_ID + ts).encode('utf-8')
        md5 = hashlib.md5()
        md5.update(tt)
        base_string = md5.hexdigest().encode('utf-8')
        signa = hmac.new(RTASR_API_KEY.encode('utf-8'), base_string, hashlib.sha1).digest()
        signa = base64.b64encode(signa).decode('utf-8')

        self.rtasr_url = f"{base_url}?appid={RTASR_APP_ID}&ts={ts}&signa={quote(signa)}"
        self.end_tag = '{"end": true}'

        try:
            self.rtasr_ws = create_connection(self.rtasr_url)
            print(f"实时语音转写连接成功: {self.rtasr_url}")
        except Exception as e:
            print(f"实时语音转写连接失败: {e}")

    def send_audio(self):
        """从麦克风实时采集并发送音频到语音转写服务"""
        if not self.rtasr_ws:
            print("语音转写未连接，无法发送")
            return
        CHUNK = 1280
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 16000

        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

        print("开始录音，按 Ctrl+C 停止...")
        try:
            while True:
                audio_data = stream.read(CHUNK)
                self.rtasr_ws.send(audio_data)
                time.sleep(0.04)
        except KeyboardInterrupt:
            self.rtasr_ws.send(self.end_tag.encode('utf-8'))
            print("语音转写发送结束标志成功")
            # 手动结束时立即处理剩余文本
            self.process_buffered_text()
        except Exception as e:
            print(f"录音或发送失败: {e}")
        finally:
            stream.stop_stream()
            stream.close()
            p.terminate()
            self.close()

    def recv_rtasr(self):
        """接收实时语音转写结果并收集纯文本"""
        if not self.rtasr_ws:
            return
        try:
            while self.rtasr_ws.connected:
                result = self.rtasr_ws.recv()
                if not result:
                    print("语音转写接收结果结束")
                    break
                    
                result_dict = json.loads(result)
                if result_dict["action"] == "started":
                    print("语音转写握手成功: " + result)
                elif result_dict["action"] == "result":
                    try:
                        # 解析 data 字段为 JSON
                        data = json.loads(result_dict["data"])
                        # 提取纯文本
                        text = ""
                        if "cn" in data and "st" in data["cn"] and "rt" in data["cn"]["st"]:
                            for rt in data["cn"]["st"]["rt"]:
                                for ws in rt["ws"]:
                                    for cw in ws["cw"]:
                                        text += cw["w"]
                            
                            print("实时转录结果: " + text)
                            # 智能更新文本缓冲区
                            self.update_text_buffer(text)
                            self.last_text_time = time.time()
                    except Exception as e:
                        print(f"解析转录结果失败: {e}")
                        
                elif result_dict["action"] == "error":
                    print("语音转写错误: " + result)
                    self.rtasr_ws.close()
                    return
        except Exception as e:
            print(f"语音转写接收异常: {e}")

    def update_text_buffer(self, new_text):
        """智能更新文本缓冲区，避免重复和优化片段拼接"""
        # 首次添加文本
        if not self.text_buffer:
            self.text_buffer.append(new_text)
            return
            
        # 检查新文本是否是之前文本的扩展版本
        last_text = self.text_buffer[-1]
        
        # 如果新文本完全包含上一个文本（是上一个的扩展），则替换
        if new_text.startswith(last_text):
            self.text_buffer[-1] = new_text
        # 如果新文本是上一个文本的变体（有部分重叠）
        elif self.is_similar(last_text, new_text):
            # 找到重叠部分并合并
            merged_text = self.merge_texts(last_text, new_text)
            self.text_buffer[-1] = merged_text
        # 如果是完全新的文本段落（通常由标点符号分隔）
        elif new_text.startswith(("。", "？", "！", "，")):
            # 可能是上一句的结束标点，尝试附加到上一个文本
            if len(new_text) <= 2:  # 仅标点符号
                if not last_text.endswith(("。", "？", "！")):
                    self.text_buffer[-1] = last_text + new_text
            else:
                self.text_buffer.append(new_text)
        # 完全新的文本
        else:
            self.text_buffer.append(new_text)

    def is_similar(self, text1, text2):
        """检查两个文本是否相似（一个是另一个的变形）"""
        # 如果两个文本长度相差太大，直接认为不相似
        if abs(len(text1) - len(text2)) > max(len(text1), len(text2)) * 0.7:
            return False
            
        # 检查是否有足够的子字符串重叠
        min_length = min(len(text1), len(text2))
        for i in range(min(min_length, 5), 0, -1):  # 从较长的子串开始检查
            for j in range(len(text1) - i + 1):
                substring = text1[j:j+i]
                if substring in text2:
                    return True
        return False

    def merge_texts(self, text1, text2):
        """智能合并两个相似的文本片段"""
        # 如果一个文本包含另一个，返回较长的
        if text1 in text2:
            return text2
        if text2 in text1:
            return text1
            
        # 查找最长的公共子串
        common = self.longest_common_substring(text1, text2)
        if not common:
            # 无法找到公共部分，简单连接
            return text1 + text2
            
        # 根据公共子串的位置拼接
        pos1 = text1.find(common)
        pos2 = text2.find(common)
        
        prefix = text1[:pos1] if pos1 < pos2 else text2[:pos2]
        suffix = text1[pos1+len(common):] if pos1+len(common) < len(text1) else text2[pos2+len(common):]
        
        return prefix + common + suffix
        
    def longest_common_substring(self, s1, s2):
        """查找两个字符串之间的最长公共子串"""
        if not s1 or not s2:
            return ""
            
        m = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
        longest, end_pos = 0, 0
        
        for i in range(1, len(s1) + 1):
            for j in range(1, len(s2) + 1):
                if s1[i-1] == s2[j-1]:
                    m[i][j] = m[i-1][j-1] + 1
                    if m[i][j] > longest:
                        longest = m[i][j]
                        end_pos = i
        
        return s1[end_pos - longest:end_pos]

    def check_silence(self):
        """定时检查沉默时间并处理文本"""
        while True:
            try:
                current_time = time.time()
                if self.text_buffer and (current_time - self.last_text_time >= SILENCE_THRESHOLD):
                    self.process_buffered_text()
                time.sleep(1)  # 每秒检查一次
            except Exception as e:
                print(f"检查沉默时间异常: {e}")
                time.sleep(1)

    def process_buffered_text(self):
        """整合并处理缓冲的文本"""
        if not self.text_buffer:
            return
            
        try:
            # 智能合并文本
            final_text = self.smart_join_texts(self.text_buffer)
            # 清理文本
            final_text = self.clean_text(final_text)
            
            print(f"整合后的完整文本: {final_text}")
            
            # 处理最终文本
            if final_text.strip():  # 确保文本不是空的
                self.nlp_process(final_text)
                
            # 清空文本缓冲区
            self.text_buffer = []
        except Exception as e:
            print(f"处理缓冲文本异常: {e}")

    def smart_join_texts(self, texts):
        """智能合并文本片段，避免重复和不连贯"""
        if not texts:
            return ""
            
        if len(texts) == 1:
            return texts[0]
            
        result = texts[0]
        
        for i in range(1, len(texts)):
            current = texts[i]
            # 如果当前文本已包含在结果中，跳过
            if current in result:
                continue
                
            # 尝试查找重叠
            overlap = False
            min_overlap_len = min(3, min(len(result), len(current)))
            
            # 检查结尾与开头的重叠
            for j in range(min(len(result), 10), min_overlap_len - 1, -1):
                if result[-j:] == current[:j]:
                    result += current[j:]
                    overlap = True
                    break
            
            # 如果没有找到重叠，根据标点符号决定如何拼接
            if not overlap:
                # 判断是否需要添加标点符号
                if result.endswith(("。", "？", "！", ".", "?", "!")):
                    result += " " + current
                else:
                    result += "，" + current
        
        return result

    def clean_text(self, text):
        """清理文本，移除重复词、标点冗余和语气词"""
        if not text:
            return ""
            
        # 1. 清理重复的标点符号
        for punct in ["。", "，", "？", "！", "、", "；", "：", ".", ",", "?", "!", ";", ":"]:
            while punct + punct in text:
                text = text.replace(punct + punct, punct)
        
        # 2. 清理重复的短语和词
        # 检测像"你你"这样的重复
        text = re.sub(r'([一-龥])\1+', r'\1', text)
        
        # 检测重复的短词组
        for length in range(2, 6):  # 检查2到5个字符的重复
            for i in range(len(text) - length * 2 + 1):
                chunk = text[i:i+length]
                if chunk == text[i+length:i+length*2]:
                    # 发现重复，删除
                    text = text[:i+length] + text[i+length*2:]
                    # 回退以检查可能的其他重复
                    i = max(0, i-length)
        
        # 3. 清理常见口语语气词和停顿词
        filler_words = ["嗯", "啊", "呃", "那个", "这个", "就是", "然后", "所以", "其实", "毕竟", "你看", "怎么说"]
        for word in filler_words:
            # 移除过多的语气词，但保留有意义的
            if text.count(word) > 2:
                indices = [i for i, _ in enumerate(text) if text[i:i+len(word)] == word]
                # 保留第一个和最后一个出现的语气词
                for idx in indices[1:-1]:
                    text = text[:idx] + text[idx+len(word):]
        
        # 4. 处理语气词连用
        text = re.sub(r'(嗯|啊|呃)\1+', r'\1', text)
        
        # 5. 智能处理标点符号
        text = re.sub(r'([。？！])[，,]', r'\1', text)  # 移除句号后的逗号
        
        return text

    def nlp_process(self, text):
        """处理纯文本，使用LLM进行回复"""
        if not text.strip():
            return
            
        print(f"正在处理文本: {text}")
        try:
            if self.llm:
                print("AI回复: ", end="", flush=True)
                for chunk in self.llm.stream(text):
                    print(chunk, end="", flush=True)
                print("\n")
                
            else:
                print("大模型未初始化，无法处理文本")
        except Exception as e:
            print(f"NLP处理异常: {e}")

    def close(self):
        """关闭连接"""
        if self.rtasr_ws and self.rtasr_ws.connected:
            self.rtasr_ws.close()
            print("实时语音转写连接已关闭")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    client = Client()
    client.send_audio()


实时语音转写连接成功: ws://rtasr.xfyun.cn/v1/ws?appid=a4ab676a&ts=1742345675&signa=ImSQza2SRd3SCYx1V0Mjzd3khVI%3D
本地大模型初始化成功
语音转写握手成功: {"action":"started","code":"0","data":"","desc":"success","sid":"rta125cdead@dx2f5f1b3561cb000100"}
开始录音，按 Ctrl+C 停止...
实时转录结果: 你
实时转录结果: 你好
整合后的完整文本: 你好
正在处理文本: 你好
AI回复: 你好！很高兴为你服务。有什么问题或需要帮助的吗？

实时转录结果: 这
实时转录结果: 咱们
实时转录结果: 。那咱们去
整合后的完整文本: 这，。那咱们去
正在处理文本: 这，。那咱们去
AI回复: 您好！看起来您的句子被打断了，可以请您完整地说一下您想去哪里或者想做什么吗？这样我可以更好地帮助到您。比如：“这，。那我们去公园吧。”或者“这，。那个会议几点开始呢？”等。

实时转录结果: 你们俩
实时转录结果: 嗯
实时转录结果: 嗯嗯嗯嗯
实时转录结果: 举
实时转录结果: 取得
实时转录结果: 取得重要
实时转录结果: 我都要
实时转录结果: 我都要看一下。我的
实时转录结果: 嗯
实时转录结果: 还有
语音转写发送结束标志成功
整合后的完整文本: 你们俩，嗯，举，我都要看一下。我的，还有
正在处理文本: 你们俩，嗯，举，我都要看一下。我的，还有
AI回复: 实时转录结果: 嗯.
语音转写接收结果结束
看起来您似乎在询问关于“我们俩”和“举”的内容，但表述中有些许不清晰。能否请您具体说明一下您想整合后的完整文本: 你们俩，嗯，举，我都要看一下。我的，还有，嗯.
正在处理文本: 你们俩，嗯，举，我都要看一下。我的，还有，嗯.
AI回复: 了解的内容或问题是什么？这样我可以更准确地帮助到您。比如，您是在问两个事物之间的比较吗？还是有关于某个特定主题的信息需求？或者是其他含义呢？请提供更多细节，我会尽力满足您的需求。



看起来您的话里有些断断续续的，可能是想表达一些具体的内容但没有完全说出来。您可以清晰地告诉我您想要了解什么吗？无论是关于产品信息、技术支持还是其他方面的问题，我都非常乐意帮助您。如果有具体的文档或内容需要查看，也可以直接告知我相关细节，我会尽力协助您。



In [5]:
# -*- encoding: utf-8 -*-
import hashlib
import hmac
import base64
import json
import time
import threading
import re
from websocket import create_connection
from urllib.parse import quote
import logging
import pyaudio
import requests
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import websocket
import datetime
from wsgiref.handlers import format_date_time
from time import mktime
import _thread as thread
import os
from urllib.parse import urlencode
# 实时语音转写配置  
RTASR_APP_ID = "a4ab676a"  
RTASR_API_KEY = "c0a8020c200794a61aaa108740ab170c" 

# 语音合成配置
TTS_APP_ID = '3b2f8d66'
TTS_API_SECRET = 'NWE0MjY1N2Y0NGU2NDUwNTQ3NzhkN2Qx'
TTS_API_KEY = '5fe3fa9b6a931c920448c48ed534543d'

# 沉默时间阈值（秒）
SILENCE_THRESHOLD = 3  

class TextToSpeech:
    STATUS_FIRST_FRAME = 0
    STATUS_CONTINUE_FRAME = 1
    STATUS_LAST_FRAME = 2

    def __init__(self, appid=TTS_APP_ID, api_key=TTS_API_KEY, api_secret=TTS_API_SECRET):
        self.APPID = appid
        self.APIKey = api_key
        self.APISecret = api_secret
        self.audio_data = bytearray()

    def _create_url(self, text):
        url = 'wss://tts-api.xfyun.cn/v2/tts'
        now = datetime.datetime.now()
        date = format_date_time(mktime(now.timetuple()))
        signature_origin = "host: ws-api.xfyun.cn\n" + "date: " + date + "\n" + "GET /v2/tts HTTP/1.1"
        signature_sha = hmac.new(self.APISecret.encode('utf-8'), signature_origin.encode('utf-8'), hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode('utf-8')
        authorization_origin = f'api_key="{self.APIKey}", algorithm="hmac-sha256", headers="host date request-line", signature="{signature_sha}"'
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode('utf-8')
        v = {"authorization": authorization, "date": date, "host": "ws-api.xfyun.cn"}
        return url + '?' + urlencode(v)

    def _on_message(self, ws, message):
        try:
            message = json.loads(message)
            code = message["code"]
            audio = base64.b64decode(message["data"]["audio"])
            status = message["data"]["status"]
            if code != 0:
                print(f"TTS error: {message['message']} code: {code}")
            else:
                self.audio_data.extend(audio)
            if status == self.STATUS_LAST_FRAME:
                ws.close()
        except Exception as e:
            print(f"TTS message parse exception: {e}")

    def _on_error(self, ws, error):
        print(f"TTS error: {error}")

    def _on_close(self, ws, *args, **kwargs):
        print("TTS WebSocket closed")

    def _on_open(self, ws):
        def run(*args):
            common_args = {"app_id": self.APPID}
            business_args = {"aue": "raw", "auf": "audio/L16;rate=16000", "vcn": "xiaoyan", "tte": "utf8"}
            data_args = {"status": self.STATUS_LAST_FRAME, "text": str(base64.b64encode(self.Text.encode('utf-8')), "UTF8")}
            d = {"common": common_args, "business": business_args, "data": data_args}
            ws.send(json.dumps(d))
        thread.start_new_thread(run, ())

    def synthesize(self, text):
        self.Text = text
        self.audio_data = bytearray()
        ws_url = self._create_url(text)
        websocket.enableTrace(False)
        ws = websocket.WebSocketApp(ws_url, on_message=self._on_message, on_error=self._on_error, on_close=self._on_close)
        ws.on_open = self._on_open
        ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})
        return bytes(self.audio_data)

class Client:
    def __init__(self):
        # 初始化实时语音转写
        self.rtasr_ws = None
        self.setup_rtasr()

        # 初始化语音合成
        self.tts = TextToSpeech()

        # 用于存储转录文本和时间戳
        self.text_buffer = []
        self.last_text_time = time.time()
        
        # 初始化大模型
        try:
            self.llm = Ollama(model="qwen2.5:7b-instruct", temperature=0.5)
            self.memory = ConversationBufferMemory()
            self.conversation = ConversationChain(llm=self.llm, memory=self.memory)
            print("本地大模型初始化成功")
        except Exception as e:
            print(f"大模型初始化失败: {e}")
            self.llm = None

        # 初始化音频播放
        self.audio_player = pyaudio.PyAudio()

        # 启动接收线程
        self.trecv_rtasr = threading.Thread(target=self.recv_rtasr)
        self.trecv_rtasr.daemon = True
        self.trecv_rtasr.start()

        # 启动定时检查线程
        self.tcheck = threading.Thread(target=self.check_silence)
        self.tcheck.daemon = True
        self.tcheck.start()

    def setup_rtasr(self):
        base_url = "ws://rtasr.xfyun.cn/v1/ws"
        ts = str(int(time.time()))
        tt = (RTASR_APP_ID + ts).encode('utf-8')
        md5 = hashlib.md5()
        md5.update(tt)
        base_string = md5.hexdigest().encode('utf-8')
        signa = hmac.new(RTASR_API_KEY.encode('utf-8'), base_string, hashlib.sha1).digest()
        signa = base64.b64encode(signa).decode('utf-8')
        self.rtasr_url = f"{base_url}?appid={RTASR_APP_ID}&ts={ts}&signa={quote(signa)}"
        self.end_tag = '{"end": true}'
        try:
            self.rtasr_ws = create_connection(self.rtasr_url)
            print(f"实时语音转写连接成功: {self.rtasr_url}")
        except Exception as e:
            print(f"实时语音转写连接失败: {e}")

    def send_audio(self):
        if not self.rtasr_ws:
            print("语音转写未连接，无法发送")
            return
        CHUNK = 1280
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 16000
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
        print("开始录音，按 Ctrl+C 停止...")
        try:
            while True:
                audio_data = stream.read(CHUNK)
                self.rtasr_ws.send(audio_data)
                time.sleep(0.04)
        except KeyboardInterrupt:
            self.rtasr_ws.send(self.end_tag.encode('utf-8'))
            print("语音转写发送结束标志成功")
            self.process_buffered_text()
        except Exception as e:
            print(f"录音或发送失败: {e}")
        finally:
            stream.stop_stream()
            stream.close()
            p.terminate()
            self.close()

    def recv_rtasr(self):
        if not self.rtasr_ws:
            return
        try:
            while self.rtasr_ws.connected:
                result = self.rtasr_ws.recv()
                if not result:
                    print("语音转写接收结果结束")
                    break
                result_dict = json.loads(result)
                if result_dict["action"] == "started":
                    print("语音转写握手成功: " + result)
                elif result_dict["action"] == "result":
                    try:
                        data = json.loads(result_dict["data"])
                        text = ""
                        if "cn" in data and "st" in data["cn"] and "rt" in data["cn"]["st"]:
                            for rt in data["cn"]["st"]["rt"]:
                                for ws in rt["ws"]:
                                    for cw in ws["cw"]:
                                        text += cw["w"]
                            print("实时转录结果: " + text)
                            self.update_text_buffer(text)
                            self.last_text_time = time.time()
                    except Exception as e:
                        print(f"解析转录结果失败: {e}")
                elif result_dict["action"] == "error":
                    print("语音转写错误: " + result)
                    self.rtasr_ws.close()
                    return
        except Exception as e:
            print(f"语音转写接收异常: {e}")

    def update_text_buffer(self, new_text):
        if not self.text_buffer:
            self.text_buffer.append(new_text)
            return
        last_text = self.text_buffer[-1]
        if new_text.startswith(last_text):
            self.text_buffer[-1] = new_text
        elif self.is_similar(last_text, new_text):
            merged_text = self.merge_texts(last_text, new_text)
            self.text_buffer[-1] = merged_text
        elif new_text.startswith(("。", "？", "！", "，")):
            if len(new_text) <= 2:
                if not last_text.endswith(("。", "？", "！")):
                    self.text_buffer[-1] = last_text + new_text
            else:
                self.text_buffer.append(new_text)
        else:
            self.text_buffer.append(new_text)

    def is_similar(self, text1, text2):
        if abs(len(text1) - len(text2)) > max(len(text1), len(text2)) * 0.7:
            return False
        min_length = min(len(text1), len(text2))
        for i in range(min(min_length, 5), 0, -1):
            for j in range(len(text1) - i + 1):
                substring = text1[j:j+i]
                if substring in text2:
                    return True
        return False

    def merge_texts(self, text1, text2):
        if text1 in text2:
            return text2
        if text2 in text1:
            return text1
        common = self.longest_common_substring(text1, text2)
        if not common:
            return text1 + text2
        pos1 = text1.find(common)
        pos2 = text2.find(common)
        prefix = text1[:pos1] if pos1 < pos2 else text2[:pos2]
        suffix = text1[pos1+len(common):] if pos1+len(common) < len(text1) else text2[pos2+len(common):]
        return prefix + common + suffix

    def longest_common_substring(self, s1, s2):
        if not s1 or not s2:
            return ""
        m = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
        longest, end_pos = 0, 0
        for i in range(1, len(s1) + 1):
            for j in range(1, len(s2) + 1):
                if s1[i-1] == s2[j-1]:
                    m[i][j] = m[i-1][j-1] + 1
                    if m[i][j] > longest:
                        longest = m[i][j]
                        end_pos = i
        return s1[end_pos - longest:end_pos]

    def check_silence(self):
        while True:
            try:
                current_time = time.time()
                if self.text_buffer and (current_time - self.last_text_time >= SILENCE_THRESHOLD):
                    self.process_buffered_text()
                time.sleep(1)
            except Exception as e:
                print(f"检查沉默时间异常: {e}")
                time.sleep(1)

    def process_buffered_text(self):
        if not self.text_buffer:
            return
        try:
            final_text = self.smart_join_texts(self.text_buffer)
            final_text = self.clean_text(final_text)
            print(f"整合后的完整文本: {final_text}")
            if final_text.strip():
                self.nlp_process(final_text)
            self.text_buffer = []
        except Exception as e:
            print(f"处理缓冲文本异常: {e}")

    def smart_join_texts(self, texts):
        if not texts:
            return ""
        if len(texts) == 1:
            return texts[0]
        result = texts[0]
        for i in range(1, len(texts)):
            current = texts[i]
            if current in result:
                continue
            overlap = False
            min_overlap_len = min(3, min(len(result), len(current)))
            for j in range(min(len(result), 10), min_overlap_len - 1, -1):
                if result[-j:] == current[:j]:
                    result += current[j:]
                    overlap = True
                    break
            if not overlap:
                if result.endswith(("。", "？", "！", ".", "?", "!")):
                    result += " " + current
                else:
                    result += "，" + current
        return result

    def clean_text(self, text):
        if not text:
            return ""
        for punct in ["。", "，", "？", "！", "、", "；", "：", ".", ",", "?", "!", ";", ":"]:
            while punct + punct in text:
                text = text.replace(punct + punct, punct)
        text = re.sub(r'([一-龥])\1+', r'\1', text)
        for length in range(2, 6):
            for i in range(len(text) - length * 2 + 1):
                chunk = text[i:i+length]
                if chunk == text[i+length:i+length*2]:
                    text = text[:i+length] + text[i+length*2:]
                    i = max(0, i-length)
        filler_words = ["嗯", "啊", "呃", "那个", "这个", "就是", "然后", "所以", "其实", "毕竟", "你看", "怎么说"]
        for word in filler_words:
            if text.count(word) > 2:
                indices = [i for i, _ in enumerate(text) if text[i:i+len(word)] == word]
                for idx in indices[1:-1]:
                    text = text[:idx] + text[idx+len(word):]
        text = re.sub(r'(嗯|啊|呃)\1+', r'\1', text)
        text = re.sub(r'([。？！])[，,]', r'\1', text)
        return text

    def play_audio(self, audio_data, rate=16000):
        """播放音频数据"""
        stream = self.audio_player.open(format=pyaudio.paInt16, channels=1, rate=rate, output=True)
        stream.write(audio_data)
        stream.stop_stream()
        stream.close()

    def nlp_process(self, text):
        if not text.strip():
            return
        print(f"正在处理文本: {text}")
        try:
            if self.llm:
                print("AI回复: ", end="", flush=True)
                response = ""
                for chunk in self.llm.stream(text):
                    print(chunk, end="", flush=True)
                    response += chunk
                print("\n")
                # 语音合成并播放
                audio_data = self.tts.synthesize(response)
                self.play_audio(audio_data)
            else:
                print("大模型未初始化，无法处理文本")
        except Exception as e:
            print(f"NLP处理异常: {e}")

    def close(self):
        if self.rtasr_ws and self.rtasr_ws.connected:
            self.rtasr_ws.close()
            print("实时语音转写连接已关闭")
        self.audio_player.terminate()

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    client = Client()
    client.send_audio()

实时语音转写连接成功: ws://rtasr.xfyun.cn/v1/ws?appid=a4ab676a&ts=1742345786&signa=J5leoVzWdvhrHsllTJe546QouvQ%3D
本地大模型初始化成功
语音转写握手成功: {"action":"started","code":"0","data":"","desc":"success","sid":"rta06271ea7@dx5bb21b35623a1aba00"}
开始录音，按 Ctrl+C 停止...
录音或发送失败: [Errno -9999] Unanticipated host error


OSError: Stream not open

语音转写错误: {"action":"error","code":"10700","data":"","desc":"engine error|37005:Client idle timeout","sid":"rta06271ea7@dx5bb21b35623a1aba00"}
